# Object Pool Pattern

Manages a pool of reusable objects to improve performance and resource utilization.

## Intent

- Reduce overhead of object creation and destruction
- Manage expensive resources efficiently
- Improve performance by reusing objects
- Control maximum number of objects

## Implementation

In [1]:
class Car:
    def __init__(self, id: int) -> None:
        self.id = id

    def reset(self) -> None:
        print(f"Resetting car {self.id}")

    def __str__(self) -> str:
        return f"Car(id={self.id})"


class Garage:
    def __init__(self, size: int) -> None:
        self._available = [Car(i) for i in range(size)]
        self._in_use = []

    def acquire(self) -> Car:
        if not self._available:
            raise Exception("No cars available")
        car = self._available.pop()
        self._in_use.append(car)
        return car

    def release(self, car: Car) -> None:
        car.reset()
        self._in_use.remove(car)
        self._available.append(car)

    def __str__(self) -> str:
        return f"Garage(available={len(self._available)}, in_use={len(self._in_use)})"

## Usage

In [2]:
if __name__ == "__main__":
    garage = Garage(2)
    print(garage)

    car1 = garage.acquire()
    print(f"Acquired: {car1}")
    print(garage)

    car2 = garage.acquire()
    print(f"Acquired: {car2}")
    print(garage)

    garage.release(car1)
    print(f"Released: {car1}")
    print(garage)

    car3 = garage.acquire()
    print(f"Acquired: {car3}")
    print(garage)

Garage(available=2, in_use=0)
Acquired: Car(id=1)
Garage(available=1, in_use=1)
Acquired: Car(id=0)
Garage(available=0, in_use=2)
Resetting car 1
Released: Car(id=1)
Garage(available=1, in_use=1)
Acquired: Car(id=1)
Garage(available=0, in_use=2)
